<a href="https://colab.research.google.com/github/thepelkus-too/notebook-test/blob/master/kanjisplorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import argparse
import sys
import numpy as np
import json
import os
import re
from os.path import isfile, join
import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions, preprocess_input
from keras.models import Model
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from scipy.spatial import distance

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [0]:
def process_arguments(args):
    parser = argparse.ArgumentParser(description='tSNE on audio')
    parser.add_argument('--images_path', action='store',
                        help='path to directory of images')
    parser.add_argument('--output_path', action='store',
                        help='path to where to put output json file')
    parser.add_argument('--num_dimensions', action='store', default=2,
                        help='dimensionality of t-SNE points (default 2)')
    parser.add_argument('--perplexity', action='store',
                        default=30, help='perplexity of t-SNE (default 30)')
    parser.add_argument('--learning_rate', action='store',
                        default=150, help='learning rate of t-SNE (default 150)')
    parser.add_argument('--saved_activations_path', action='store',
                        help='path to saved activations, or where to save if not saved')
    params = vars(parser.parse_args(args))
    return params

In [0]:
def get_image(path, input_shape):
    img = image.load_img(path, target_size=input_shape)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x


In [0]:
def find_candidate_images(images_path):
    """
    Finds all candidate images in the given folder and its sub-folders.

    Returns:
        images: a list of absolute paths to the discovered images.
    """
    images = []
    for root, dirs, files in os.walk(images_path):
        for name in files:
            file_path = os.path.abspath(os.path.join(root, name))
            if ((os.path.splitext(name)[1]).lower() in ['.jpg', '.png', '.jpeg']):
                images.append(file_path)
    return images


In [0]:
def analyze_images(images_path, saved_activations_path):
    # try to get activations from path first
    activations_load_path = f'{saved_activations_path}_act.npy'
    images_load_path = f'{saved_activations_path}_img.npy'
    if isfile(activations_load_path) and isfile(images_load_path):
        activations = np.load(activations_load_path)
        images = np.load(images_load_path)
    else:
        # make feature_extractor
        model = keras.applications.VGG16(weights='imagenet', include_top=True)
        feat_extractor = Model(
            inputs=model.input, outputs=model.get_layer("fc2").output)
        input_shape = model.input_shape[1:3]
        # get images
        candidate_images = find_candidate_images(images_path)
        # analyze images and grab activations
        activations = []
        images = []
        combined = []
        for idx, image_path in enumerate(candidate_images):
            file_path = join(images_path, image_path)
            img = get_image(file_path, input_shape)
            if img is not None:
                print("getting activations for %s %d/%d" %
                      (image_path, idx, len(candidate_images)))
                acts = feat_extractor.predict(img)[0]
                activations.append(acts)
                images.append(image_path)
                combined.append(
                    {"imagepath": image_path, "activations": activations})

        # cache activations
        # TODO: use data path here
        np.save(saved_activations_path + "_act", activations)
        np.save(saved_activations_path + "_img", images)

    return images, activations

In [0]:
def pca_from_activations(images, activations, dimensions=300):
    # run PCA firt
    print("Running PCA on %d images..." % len(activations))
    features = np.array(activations)

    pca_pipeline = make_pipeline(
        StandardScaler(), PCA(n_components=dimensions))
    # pca = PCA(n_components=dimensions)
    pca_pipeline.fit(features)
    pca_features = pca_pipeline.transform(features)
    return images, pca_features


In [0]:
def kanji_for_index(kanjis, images, i):
    image_filename = images[i]
    index_from_image_filename = int(
        re.search("(\d{4})", image_filename).groups()[0])

    return kanjis[index_from_image_filename]


In [0]:
def nearest_kanji(full_data, images, kanjis, start, end, num_neighbors):
    working_data = full_data[start:end]
    nbrs = NearestNeighbors(n_neighbors=num_neighbors,
                            algorithm='ball_tree').fit(working_data)
    distances, indices = nbrs.kneighbors(working_data)
    neighbors = []
    for indices_x in indices:
        try:
            chars = [kanji_for_index(kanjis, images[start:end], x)
                     for x in indices_x]
            neighbors = neighbors + [chars]
        except:
            images_nums = np.array([images[start:end][x] for x in indices_x])
            print(f'BORKED {indices_x} {images_nums}')

    return neighbors


In [0]:
def run_tsne(images_path, output_path, tsne_dimensions, tsne_perplexity, tsne_learning_rate, saved_activations_path):
    images, activations = analyze_images(images_path, saved_activations_path)

    f = open('kanjis.json')
    kanjis = json.load(f)

    # start = 0
    # end = len(activations)

    # activations_nn = nearest_kanji(activations, images, kanjis, start, end, 7)
    # for ann in activations_nn:
    #     print(ann)

    # sys.exit()

    images, pca_features_2d = pca_from_activations(images, activations, 2)
    
    # pca_nn = nearest_kanji(pca_features_2d, images, kanjis, start, end, 7)

    for (ann, pcann) in zip(activations_nn, pca_nn):
        print(ann)
        print(pcann)

    # Let's look at X axis
    # for entry in sorted(filter(lambda x: x[1][1] > -1 and x[1][1] < 1, enumerate(pca_features_2d)), key=lambda x: x[1][0]):
    #     x = entry[1][0]
    #     y = entry[1][1]
    #     i = entry[0]
    #     kanji = kanji_for_index(kanjis, images, i)
    #     print(f'{x} {y} {kanji}')

    # Let's look at Y axis
    # for entry in sorted(filter(lambda x: x[1][0] > -1 and x[1][0] < 1, enumerate(pca_features_2d)), key=lambda x: x[1][1]):
    #     x = entry[1][0]
    #     y = entry[1][1]
    #     i = entry[0]
    #     kanji = kanji_for_index(kanjis, images, i)
    #     print(f'{x} {y} {kanji}')

    sys.exit()

    images, pca_features = pca_from_activations(
        images, activations)
    print("Running t-SNE on %d images..." % len(images))
    X = np.array(pca_features)
    tsne = TSNE(n_iter=2000, n_components=tsne_dimensions, learning_rate=tsne_learning_rate,
                perplexity=tsne_perplexity, verbose=2).fit_transform(X)
    # save data to json
    data = []
    for i, f in enumerate(images):
        point = [float((tsne[i, k] - np.min(tsne[:, k]))/(np.max(tsne[:, k]
                                                                 ) - np.min(tsne[:, k]))) for k in range(tsne_dimensions)]
        data.append({"path": os.path.abspath(
            join(images_path, images[i])), "point": point})
    with open(output_path, 'w') as outfile:
        json.dump(data, outfile)

In [0]:
images_path = "./jpg"
output_path = "tsneCoords.json"
tsne_dimensions = 3
tsne_perplexity = 50
tsne_learning_rate = 150
saved_activations_path = "full"

In [0]:
run_tsne(images_path,
         output_path,
         tsne_dimensions,
         tsne_perplexity,
         tsne_learning_rate,
         saved_activations_path)

553467904/553467096 [==============================] - 8s 0us/step


FileNotFoundError: ignored